In [1]:
!pip install lxml

In [2]:
!pip install html5lib

In [3]:
!pip install beautifulsoup4

# Segmenting and Clustering Neighborhoods in Toronto

In [4]:
import pandas as pd
import numpy as np
import html5lib
import lxml
import bs4
import matplotlib.pyplot as plt
from IPython.display import Image, HTML
%matplotlib inline

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

In [5]:
#To build the code to scrape data from the Wikipedia page.
d = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [6]:
#Determine numbers of table imported
len(d)

3

## Part 1: To create the dataframe from Wikipedia

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood  <br/>
- Only process the cells that have an assigned borough. Ignore cells with a borough that is <b> Not assigned</b> .<br/>
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.<br/>
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.<br/>
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.<br/>
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [7]:
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df = pd.DataFrame(d[0])
df = df.rename(columns={'Postcode': 'Postalcode'})

df.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [8]:
#To filter 'Not assigned' row from Borough
df_b = df[df.Borough !='Not assigned']

df_b.head()

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [9]:
#To groupby 'Postalcode' & 'Borough' to combine 'Neighbourhood'
df = df_b.groupby(['Postalcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
print ("The number of rows in my dataframe is", df.shape)

The number of rows in my dataframe is (103, 3)


## Part 2: To create dataframe with geographical coordinates

In [11]:
! pip install geocoder 
import geocoder

In [13]:
import requests
import io
url="http://cocl.us/Geospatial_data"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [14]:
# rename the first column to allow merging dataframes on Postcode
c.columns = ['Postalcode', 'Latitude', 'Longitude']
df = pd.merge(c, df, on='Postalcode')

# reorder column names and show the dataframe
df = df[['Postalcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]
df.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Part 3: To generate maps to visualize your neighborhoods and how they cluster together.

In [15]:
# create map of New York using latitude and longitude values
import folium

latitude = 43.700110
longitude = -79.347015

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='coral',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto